In [10]:
# Import useful libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Ignore warnings
import warnings
warnings.simplefilter('ignore')

# Import evaluation libraries
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Random seed for reproducibility
seed = 202
np.random.seed(seed)

In [11]:
# Import the train set and test set
train_data = pd.read_csv("final_train.csv", delimiter=",")
test_data = pd.read_csv("final_test.csv", delimiter=",")

# Drop the ID column
train_data = train_data.drop('ID', axis=1)

# Sort the dataset
train_data = train_data.iloc[np.random.permutation(len(train_data))]
train_data = train_data.reset_index(drop=True)

In [12]:
array = np.asarray(train_data)

# X,Y are the splits between features and labels used to evaluate SelectKBest
X = array[:,0:train_data.shape[1]-1]
X = np.asarray(X)
Y = array[:,train_data.shape[1]-1]
Y = np.asarray(Y)

# Evaluate the features with a chi2 test by using SelectKBest
# It must be at most max_feature_number - 1
feature_number = 29

chi2_test = SelectKBest(score_func=chi2, k=feature_number)
fit = chi2_test.fit(X,Y)

In [13]:
def features_labels_split(data):
    
    features = data[train_data.columns[chi2_test.get_support(indices=True)]]
    labels = data['Product']
    return features, labels

In [14]:
def standardize_features(features):

    scaler = MinMaxScaler().fit(features)
    features = scaler.transform(features)
    return features

In [15]:
def train_test_validation_split(features, labels, val_samples, test_samples):

    features = np.asarray(features)
    features = standardize_features(features)
    labels =np.asarray(labels)
    
    X_test = features[0:test_samples]
    y_test = labels[0:test_samples]

    X_val = features[test_samples:test_samples + val_samples]
    y_val = labels[test_samples:test_samples + val_samples]

    X_train = features[test_samples + val_samples:]
    y_train = labels[test_samples + val_samples:]

    return X_train, X_test, X_val, y_train, y_test, y_val

In [16]:
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras import backend as K
from keras.callbacks import EarlyStopping, TensorBoard

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [17]:
train_feat, train_label = features_labels_split(train_data)
train_label = keras.utils.to_categorical(train_label, 4)

num_val_samples = 950
num_test_samples = 950
X_train, X_test, X_val, y_train, y_test, y_val = train_test_validation_split(train_feat, train_label, num_val_samples, num_test_samples)

In [18]:
X_train.shape

(7667, 29)

In [22]:
tensorboard = TensorBoard(log_dir='Graph', histogram_freq=0, write_graph=True, write_images=False)

neurons = 64
dropout = 0.3
batch_size = 64         
epochs = 5000


output_activation_function = 'softmax'
activation_function = 'relu'

loss = 'categorical_crossentropy'
learning_rate = 0.0025
optimizer= Adam(learning_rate)

def build_model(inputs, output_size, neurons, activ_func=activation_function,
                dropout = dropout, loss=loss, optimizer=optimizer):
    
    model = Sequential()
    model.add(Dense(neurons, input_shape=(X_train.shape[1],)))
    model.add(Activation(activation_function))
    model.add(Dropout(dropout))
    model.add(Dense(neurons))
    model.add(Activation(activation_function))
    model.add(Dense(units=output_size, activation=output_activation_function))    
    model.compile(loss=loss, optimizer=optimizer, metrics=[f1])
    return model

In [20]:
# initialise model architecture
output_size = 4

model = build_model(X_train, output_size=output_size, neurons=neurons)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                1920      
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
activation_2 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
Total params: 6,340
Trainable params: 6,340
Non-trainable params: 0
_________________________________________________________________


In [23]:
# train model on data
history = model.fit(X_train, y_train,
                            epochs=epochs, batch_size=batch_size,
                            verbose=1, validation_data=(X_val, y_val),
                            callbacks=[tensorboard], shuffle=False)

Train on 7667 samples, validate on 950 samples
Epoch 1/5000
7667/7667 [==============================] - 1s 77us/step - loss: 0.8643 - f1: 0.6215 - val_loss: 1.1287 - val_f1: 0.5670
Epoch 2/5000
7667/7667 [==============================] - 1s 78us/step - loss: 0.8507 - f1: 0.6305 - val_loss: 1.1337 - val_f1: 0.5747
Epoch 3/5000
7667/7667 [==============================] - 1s 82us/step - loss: 0.8579 - f1: 0.6313 - val_loss: 1.1272 - val_f1: 0.5880
Epoch 4/5000
7667/7667 [==============================] - 1s 77us/step - loss: 0.8603 - f1: 0.6277 - val_loss: 1.1334 - val_f1: 0.5740
Epoch 5/5000
7667/7667 [==============================] - 1s 80us/step - loss: 0.8546 - f1: 0.6290 - val_loss: 1.1252 - val_f1: 0.5802
Epoch 6/5000
7667/7667 [==============================] - 1s 83us/step - loss: 0.8597 - f1: 0.6231 - val_loss: 1.1277 - val_f1: 0.5752
Epoch 7/5000
7667/7667 [==============================] - 1s 81us/step - loss: 0.8568 - f1: 0.6296 - val_loss: 1.1261 - val_f1: 0.5769
Epoch 8/

Epoch 61/5000
7667/7667 [==============================] - 1s 91us/step - loss: 0.8366 - f1: 0.6408 - val_loss: 1.1674 - val_f1: 0.5788
Epoch 62/5000
7667/7667 [==============================] - 1s 88us/step - loss: 0.8389 - f1: 0.6364 - val_loss: 1.1765 - val_f1: 0.5686
Epoch 63/5000
7667/7667 [==============================] - 1s 82us/step - loss: 0.8410 - f1: 0.6334 - val_loss: 1.1661 - val_f1: 0.5742
Epoch 64/5000
7667/7667 [==============================] - 1s 84us/step - loss: 0.8335 - f1: 0.6355 - val_loss: 1.1615 - val_f1: 0.5737
Epoch 65/5000
7667/7667 [==============================] - 1s 77us/step - loss: 0.8358 - f1: 0.6416 - val_loss: 1.1617 - val_f1: 0.5687
Epoch 66/5000
7667/7667 [==============================] - 1s 76us/step - loss: 0.8427 - f1: 0.6360 - val_loss: 1.1659 - val_f1: 0.5727
Epoch 67/5000
7667/7667 [==============================] - 1s 74us/step - loss: 0.8342 - f1: 0.6400 - val_loss: 1.1676 - val_f1: 0.5756
Epoch 68/5000
7667/7667 [=======================

KeyboardInterrupt: 